# Sales Forecasting and Customer Segmentation

In [ ]:
# Sales Forecasting and Customer Segmentation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import os # Added for directory creation

warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('ggplot')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Define output directories
plot_output_dir = 'notebooks/output_plots_forecast_segment'
data_output_dir = 'data/processed'
os.makedirs(plot_output_dir, exist_ok=True)
os.makedirs(data_output_dir, exist_ok=True)


# Load the datasets
try:
    retail_df = pd.read_csv('data/raw/retail_sales_data.csv')
    print("Loaded data/raw/retail_sales_data.csv")
    # reviews_df = pd.read_csv('data/raw/product_reviews.csv') # Loaded but not used
except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    print("Please ensure 'data/raw/retail_sales_data.csv' exists.")
    exit()

# Convert date column to datetime
retail_df['date'] = pd.to_datetime(retail_df['date'])

# Add temporal features
retail_df['year'] = retail_df['date'].dt.year
retail_df['month'] = retail_df['date'].dt.month
retail_df['day'] = retail_df['date'].dt.day
retail_df['day_of_week'] = retail_df['date'].dt.dayofweek
retail_df['is_weekend'] = retail_df['day_of_week'].isin([5, 6]).astype(int)
retail_df['quarter'] = retail_df['date'].dt.quarter

# Handle missing values
print("\nHandling missing values...")
# Fill missing weather with mode
retail_df['weather'] = retail_df['weather'].fillna(retail_df['weather'].mode()[0])
# Fill missing promotion with 'None'
retail_df['promotion'] = retail_df['promotion'].fillna('None')
# Fill missing special_event with False
retail_df['special_event'] = retail_df['special_event'].fillna(False).astype(str) # Ensure consistent type for OHE
# Fill missing dominant_age_group with mode
retail_df['dominant_age_group'] = retail_df['dominant_age_group'].fillna(retail_df['dominant_age_group'].mode()[0])
# Fill missing numerical values with median
for col in ['num_customers', 'total_sales', 'online_sales', 'in_store_sales', 'avg_transaction', 'return_rate']:
    if col in retail_df.columns: # Check if column exists
        retail_df[col] = retail_df[col].fillna(retail_df[col].median())

print("Missing values check after imputation:")
print(retail_df.isnull().sum().loc[lambda x: x > 0]) # Show only columns still having NaNs

# Feature Engineering
print("\nPerforming feature engineering...")

# Create lag features for time series
def create_lag_features(df, group_cols, target_col, lag_periods):
    df_copy = df.copy()
    for lag in lag_periods:
        df_copy[f'{target_col}_lag_{lag}'] = df_copy.groupby(group_cols)[target_col].shift(lag)
    return df_copy

# Create rolling window features
def create_rolling_features(df, group_cols, target_col, windows, functions):
    df_copy = df.copy()
    for window in windows:
        for func in functions:
            func_name = func.__name__ if hasattr(func, '__name__') else str(func)
            df_copy[f'{target_col}_roll_{window}_{func_name}'] = df_copy.groupby(group_cols)[target_col].transform(
                lambda x: x.shift(1).rolling(window=window, min_periods=1).agg(func))
    return df_copy

# Sort by date for proper time series analysis
retail_df = retail_df.sort_values(['store_id', 'category', 'date'])

# Create lag features for total_sales (1, 7, and 14 days)
retail_df = create_lag_features(retail_df, ['store_id', 'category'], 'total_sales', [1, 7, 14])

# Create rolling window features (7, 14, and 30 days with mean and std)
retail_df = create_rolling_features(retail_df, ['store_id', 'category'], 'total_sales', [7, 14, 30], [np.mean, np.std])

# Create interaction features - handle potential division by zero
retail_df['price_per_customer'] = (retail_df['total_sales'] / retail_df['num_customers']).replace([np.inf, -np.inf], 0).fillna(0)
retail_df['online_ratio'] = (retail_df['online_sales'] / retail_df['total_sales']).replace([np.inf, -np.inf], 0).fillna(0)
retail_df['weekend_promotion'] = retail_df['is_weekend'] * (retail_df['promotion'] != 'None').astype(int)

# Drop rows with NaN values created by lag/roll features
initial_rows = len(retail_df)
retail_df = retail_df.dropna(subset=[col for col in retail_df.columns if 'lag' in col or 'roll' in col])
print(f"Dropped {initial_rows - len(retail_df)} rows due to NaNs from lag/roll features.")

print("Feature engineering completed. New shape:", retail_df.shape)

# Part 1: Sales Forecasting
print("\n=== Sales Forecasting Model ===")

# Prepare data for forecasting
# Select features for forecasting
forecast_features = [
    'month', 'day', 'day_of_week', 'is_weekend', 'quarter',
    'weather', 'promotion', 'special_event', 'dominant_age_group',
    'num_customers', 'online_sales', 'in_store_sales', 'avg_transaction',
    'total_sales_lag_1', 'total_sales_lag_7', 'total_sales_lag_14',
    'total_sales_roll_7_mean', 'total_sales_roll_14_mean', 'total_sales_roll_30_mean',
    'total_sales_roll_7_std', 'total_sales_roll_14_std', 'total_sales_roll_30_std',
    'price_per_customer', 'online_ratio', 'weekend_promotion'
]
# Ensure all selected features exist in the dataframe
forecast_features = [f for f in forecast_features if f in retail_df.columns]

# Target variable
target = 'total_sales'

# Prepare categorical features
cat_features = ['weather', 'promotion', 'special_event', 'dominant_age_group']
cat_features = [f for f in cat_features if f in forecast_features] # Ensure they are in the selected features
num_features = [f for f in forecast_features if f not in cat_features]

# Create a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features) # Use sparse_output=False for XGBoost feature names
    ], remainder='passthrough') # Keep other columns if any

# Create a time series split for validation
tscv = TimeSeriesSplit(n_splits=3)

# Function to evaluate forecasting model
def evaluate_forecast_model(X, y, model, cv):
    mae_scores, rmse_scores, r2_scores = [], [], []
    # Separate preprocessor and regressor from the pipeline
    preprocessor = model.named_steps['preprocessor']
    regressor = model.named_steps['regressor']

    for fold, (train_idx, test_idx) in enumerate(cv.split(X)):
        print(f"  Fold {fold+1}/{cv.get_n_splits()}...")
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        try:
            # Fit preprocessor on training data and transform both train and test
            X_train_processed = preprocessor.fit_transform(X_train)
            X_test_processed = preprocessor.transform(X_test)
            # Fit regressor on processed training data
            regressor.fit(X_train_processed, y_train)
            y_pred = regressor.predict(X_test_processed)
            mae_scores.append(mean_absolute_error(y_test, y_pred))
            rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
            r2_scores.append(r2_score(y_test, y_pred))
        except Exception as e:
            print(f"    Error in fold {fold+1}: {e}")
            mae_scores.append(np.nan)
            rmse_scores.append(np.nan)
            r2_scores.append(np.nan)
    return {'MAE': np.nanmean(mae_scores), 'RMSE': np.nanmean(rmse_scores), 'R2': np.nanmean(r2_scores)}

# Select a sample store and category for demonstration
store_id = 'store_1'
category = 'Electronics'
sample_data = retail_df[(retail_df['store_id'] == store_id) & (retail_df['category'] == category)].copy()

if sample_data.empty:
    print(f"No data found for {store_id}, {category} after processing. Skipping forecast.")
    forecast_metrics = {'MAE': np.nan, 'RMSE': np.nan, 'R2': np.nan}
    importance_df = pd.DataFrame(columns=['Feature', 'Importance']) # Empty df
else:
    X = sample_data[forecast_features]
    y = sample_data[target]

    # Create and evaluate XGBoost model
    xgb_model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, objective='reg:squarederror'))
    ])

    print(f"\nTraining forecasting model for {store_id}, {category}...")
    forecast_metrics = evaluate_forecast_model(X, y, xgb_model, tscv)
    print("\nForecasting Model Metrics:")
    for metric, value in forecast_metrics.items():
        print(f"  {metric}: {value:.4f}")

    # Feature importance
    try:
        # Fit on the entire sample data to get importance
        xgb_model.fit(X, y)
        feature_importance = xgb_model.named_steps['regressor'].feature_importances_

        # Get feature names after one-hot encoding
        # Access the fitted OneHotEncoder to get feature names
        ohe_feature_names = list(xgb_model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(cat_features))
        # Combine numerical and OHE feature names
        all_feature_names = num_features + ohe_feature_names

        # Create feature importance DataFrame
        importance_df = pd.DataFrame({
            'Feature': all_feature_names,
            'Importance': feature_importance
        }).sort_values('Importance', ascending=False)

        # Plot feature importance
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=importance_df.head(15))
        plt.title(f'Top 15 Feature Importance for Sales Forecasting ({store_id}, {category})')
        plt.tight_layout()
        plot_path = os.path.join(plot_output_dir, 'forecast_feature_importance.png')
        plt.savefig(plot_path)
        plt.close()
        print(f"Saved plot: {plot_path}")

    except Exception as e:
        print(f"Error generating feature importance: {e}")
        importance_df = pd.DataFrame(columns=['Feature', 'Importance']) # Empty df

    # Visualize actual vs predicted values (last 30 days of sample)
    if len(X) > 30:
        split_point = len(X) - 30
        X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
        y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]
        try:
            # Explicitly fit preprocessor and transform
            preprocessor_plot = xgb_model.named_steps['preprocessor']
            regressor_plot = xgb_model.named_steps['regressor']
            X_train_processed_plot = preprocessor_plot.fit_transform(X_train)
            X_test_processed_plot = preprocessor_plot.transform(X_test)
            regressor_plot.fit(X_train_processed_plot, y_train) # Fit regressor on processed train data
            y_pred = regressor_plot.predict(X_test_processed_plot) # Predict on processed test data

            plt.figure(figsize=(12, 6))
            # Use date from original df for index if possible
            test_dates = sample_data['date'].iloc[split_point:]
            plt.plot(test_dates, y_test.values, label='Actual', marker='o', linestyle='-')
            plt.plot(test_dates, y_pred, label='Predicted', marker='x', linestyle='--')
            plt.title(f'Actual vs Predicted Sales ({store_id}, {category}) - Last 30 Days')
            plt.xlabel('Date')
            plt.ylabel('Total Sales')
            plt.legend()
            plt.xticks(rotation=45)
            plt.tight_layout()
            plot_path = os.path.join(plot_output_dir, 'forecast_actual_vs_predicted.png')
            plt.savefig(plot_path)
            plt.close()
            print(f"Saved plot: {plot_path}")
        except Exception as e:
            print(f"Error generating actual vs predicted plot: {e}")
    else:
        print("Not enough data points (<30) to generate actual vs predicted plot.")


# Part 2: Customer Segmentation
print("\n=== Customer Segmentation Analysis ===")

# Aggregate data at the store level for segmentation
store_agg_funcs = {
    'total_sales': 'mean', 'online_sales': 'mean', 'in_store_sales': 'mean',
    'num_customers': 'mean', 'avg_transaction': 'mean', 'return_rate': 'mean',
    'online_ratio': 'mean', 'price_per_customer': 'mean', 'is_weekend': 'mean'
}
# Filter functions based on columns existing in retail_df
valid_agg_funcs = {k: v for k, v in store_agg_funcs.items() if k in retail_df.columns}

if valid_agg_funcs:
    store_features = retail_df.groupby('store_id').agg(valid_agg_funcs).reset_index()

    # Calculate additional KPIs if possible
    if 'online_sales' in store_features.columns and 'in_store_sales' in store_features.columns:
        store_features['online_to_instore_ratio'] = (store_features['online_sales'] / store_features['in_store_sales']).replace([np.inf, -np.inf], 0).fillna(0)

    # Prepare data for clustering
    cluster_feature_cols = [col for col in store_features.columns if col != 'store_id']
    X_cluster = store_features[cluster_feature_cols]

    if not X_cluster.empty:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_cluster)

        # Determine optimal number of clusters using silhouette score
        silhouette_scores = []
        K = range(2, min(6, len(store_features))) # Limit K range
        print(f"\nCalculating silhouette scores for K={list(K)}...")
        for k in K:
            if k < len(X_scaled): # Ensure k is less than n_samples
                try:
                    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10) # Set n_init explicitly
                    cluster_labels = kmeans.fit_predict(X_scaled)
                    # Need at least 2 unique labels for silhouette score
                    if len(np.unique(cluster_labels)) > 1:
                         score = silhouette_score(X_scaled, cluster_labels)
                         silhouette_scores.append(score)
                         print(f"  K={k}, Silhouette Score: {score:.4f}")
                    else:
                         print(f"  K={k}, Only 1 cluster found, skipping silhouette score.")
                         silhouette_scores.append(-1) # Assign a low score
                except Exception as e:
                    print(f"  Error calculating silhouette for K={k}: {e}")
                    silhouette_scores.append(-1) # Assign a low score
            else:
                print(f"  Skipping K={k} (>= number of samples)")
                silhouette_scores.append(-1)

        # Plot silhouette scores if any were calculated
        if any(s > -1 for s in silhouette_scores):
            plt.figure(figsize=(10, 6))
            valid_K = [k for i, k in enumerate(K) if silhouette_scores[i] > -1]
            valid_scores = [s for s in silhouette_scores if s > -1]
            if valid_K:
                plt.plot(valid_K, valid_scores, 'bo-')
                plt.xlabel('Number of clusters (K)')
                plt.ylabel('Silhouette Score')
                plt.title('Silhouette Score Method For Optimal K')
                plt.grid(True)
                plot_path = os.path.join(plot_output_dir, 'optimal_clusters.png')
                plt.savefig(plot_path)
                plt.close()
                print(f"Saved plot: {plot_path}")

                # Choose optimal number of clusters
                optimal_k = valid_K[np.argmax(valid_scores)]
                print(f"Optimal number of clusters based on Silhouette Score: {optimal_k}")
            else:
                print("Could not determine optimal K from silhouette scores.")
                optimal_k = 2 # Default to 2 clusters if calculation failed
                print(f"Defaulting to K={optimal_k}")

        else:
            print("Could not calculate any valid silhouette scores.")
            optimal_k = 2 # Default to 2 clusters
            print(f"Defaulting to K={optimal_k}")


        # Apply K-means clustering with optimal k
        kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        store_features['cluster'] = kmeans.fit_predict(X_scaled)
        print(f"\nApplied K-Means with K={optimal_k}.")

        # Analyze clusters
        cluster_analysis = store_features.groupby('cluster')[cluster_feature_cols].mean().reset_index()
        print("\nCluster Analysis (Mean Values):")
        print(cluster_analysis)

        # Visualize clusters (Example: Online Ratio vs Price Per Customer)
        if 'online_ratio' in store_features.columns and 'price_per_customer' in store_features.columns:
            plt.figure(figsize=(12, 8))
            sns.scatterplot(data=store_features, x='online_ratio', y='price_per_customer', hue='cluster', palette='viridis', s=100, legend='full')
            # Add store labels
            for i, row in store_features.iterrows():
                plt.text(row['online_ratio'] + 0.001, row['price_per_customer'], row['store_id'], fontsize=9)
            plt.xlabel('Online Sales Ratio')
            plt.ylabel('Average Price Per Customer')
            plt.title(f'Store Clusters (K={optimal_k})')
            plt.legend(title='Cluster')
            plt.grid(True)
            plot_path = os.path.join(plot_output_dir, 'store_clusters.png')
            plt.savefig(plot_path)
            plt.close()
            print(f"Saved plot: {plot_path}")
        else:
            print("Skipping cluster scatter plot (missing required columns).")


        # Radar chart for cluster profiles
        def radar_chart(cluster_data_means, features_for_radar, optimal_k):
            # Normalize the mean data for radar chart (0-1 scaling often works well here)
            from sklearn.preprocessing import MinMaxScaler
            scaler_radar = MinMaxScaler()
            radar_data_scaled = pd.DataFrame(scaler_radar.fit_transform(cluster_data_means[features_for_radar]), columns=features_for_radar)

            N = len(features_for_radar)
            angles = [n / float(N) * 2 * np.pi for n in range(N)]
            angles += angles[:1]

            fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
            plt.xticks(angles[:-1], features_for_radar, color='grey', size=10)
            ax.set_rlabel_position(0)
            plt.yticks(np.linspace(0, 1, 5), [f"{i:.1f}" for i in np.linspace(0, 1, 5)], color="grey", size=8)
            plt.ylim(0, 1)

            # Plot each cluster
            for i in range(optimal_k):
                values = radar_data_scaled.iloc[i].values.flatten().tolist()
                values += values[:1]
                ax.plot(angles, values, linewidth=2, linestyle='solid', label=f"Cluster {i}")
                ax.fill(angles, values, alpha=0.2)

            plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
            plt.title('Cluster Profiles (Normalized)', size=15, y=1.1)
            return fig

        # Select features for radar chart (ensure they exist)
        radar_features = [
            'total_sales', 'online_ratio', 'num_customers', 'avg_transaction',
            'return_rate', 'price_per_customer'
        ]
        radar_features = [f for f in radar_features if f in cluster_analysis.columns]

        if len(radar_features) >= 3: # Need at least 3 features for radar
            print("\nGenerating cluster profile radar chart...")
            try:
                radar_fig = radar_chart(cluster_analysis, radar_features, optimal_k)
                plot_path = os.path.join(plot_output_dir, 'cluster_profiles_radar.png')
                radar_fig.savefig(plot_path, bbox_inches='tight') # Use bbox_inches
                plt.close(radar_fig)
                print(f"Saved plot: {plot_path}")
            except Exception as e:
                print(f"Error generating radar chart: {e}")
        else:
            print("Skipping radar chart (need at least 3 valid features).")


        # Create cluster profile descriptions
        cluster_profiles = []
        print("\nCluster Profiles:")
        for i in range(optimal_k):
            profile_desc = f"Cluster {i}: "
            cluster_mean_values = cluster_analysis.iloc[i]
            # Simple descriptions based on comparison to overall mean
            for col in radar_features: # Use features from radar chart for description
                 if col in cluster_mean_values and col in store_features.columns:
                     overall_mean = store_features[col].mean()
                     cluster_mean = cluster_mean_values[col]
                     if cluster_mean > overall_mean * 1.1: # Significantly higher
                         profile_desc += f"High {col.replace('_', ' ')}, "
                     elif cluster_mean < overall_mean * 0.9: # Significantly lower
                         profile_desc += f"Low {col.replace('_', ' ')}, "
            profile_desc = profile_desc.strip(', ') + "."
            stores_in_cluster = store_features[store_features['cluster'] == i]['store_id'].tolist()
            stores_str = ", ".join(stores_in_cluster)
            print(profile_desc)
            print(f"   Stores: {stores_str}")
            cluster_profiles.append(profile_desc) # Store description for report
            cluster_profiles.append(f"   Stores: {stores_str}") # Store stores for report


        # Save clustered data
        cluster_data_path = os.path.join(data_output_dir, 'store_clusters.csv')
        store_features.to_csv(cluster_data_path, index=False)
        print(f"\nSaved clustered store features: {cluster_data_path}")

    else:
        print("Skipping clustering due to empty feature set.")
        optimal_k = 0 # Indicate clustering was skipped
        cluster_profiles = ["Clustering skipped due to lack of data."]

else:
    print("Skipping clustering due to missing aggregated features.")
    optimal_k = 0 # Indicate clustering was skipped
    cluster_profiles = ["Clustering skipped due to missing aggregated features."]


# Final Summary
print("\n--- Script Finished ---")
print("Sales Forecasting and Customer Segmentation Analysis Completed")
if not sample_data.empty:
    print(f"Forecasting model for {store_id}, {category} achieved R² of {forecast_metrics.get('R2', np.nan):.4f}")
if optimal_k > 0:
    print(f"Stores were segmented into {optimal_k} distinct clusters")
print(f"Visualizations saved to: {plot_output_dir}")
print(f"Clustered store data saved to: {data_output_dir}")
